# Tutorial: Using Cohere with the elastic open inference API

## TODO [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()

This tutorial shows you how to compute embeddings with
Cohere using the elastic open inference API and store them for efficient vector or hybrid
search in Elasticsearch. This tutorial uses the Python Elasticsearch client
to perform the operations.

You'll learn how to:
* create an inference endpoint for text embedding using the Cohere service,
* create the necessary index mapping for the Elasticsearch index using semantic search,
* rerank with retrievers using Cohere's rerank model,
* design a RAG system with Cohere's Chat API.

The tutorial uses the [gqd/fictional-characters](https://huggingface.co/datasets/gqd/fictional-characters/blob/main/heros.json) data set.

Refer to [Cohere's tutorial](https://docs.cohere.com/docs/elasticsearch-and-cohere) for an example using a different data set.

## 🧰 Requirements

For this example, you will need:

- An Elastic deployment:
   - We'll be using [Elastic serverless](https://www.elastic.co/docs/current/serverless) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))
   
- A paid [Cohere account](https://cohere.com/) is required to use the Open Inference API with 
the Cohere service as the Cohere free trial API usage is limited.

- Python 3.7+.

- the 8.15+ python elasticsearch client

## Install and import required packages

Install Elasticsearch and Cohere:

In [10]:
!pip install elasticsearch==8.15


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


Import the required packages:

In [11]:
from elasticsearch import Elasticsearch, helpers
import json
import requests
from getpass import getpass

## Create an Elasticsearch client

Now you can instantiate the Python Elasticsearch client.

First provide your password and Cloud ID.
Then create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [13]:
ELASTIC_SERVERLESS_ENDPOINT = getpass("Elastic serverless endpoint: ")
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic API key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    endpoint=ELASTIC_SERVERLESS_ENDPOINT,
    api_key=ELASTIC_API_KEY,
)

# Confirm the client has connected
print(client.info())

TypeError: Elasticsearch.__init__() got an unexpected keyword argument 'endpoint'

## Create the inference endpoints

Create the inference endpoint first. In this example, the inference endpoint 
uses Cohere's `embed-english-v3.0` model and the `embedding_type` is set to
`byte`.

In [ ]:
COHERE_API_KEY = getpass("Cohere API key: ")
co = cohere.Client(api_key=COHERE_API_KEY)

In [ ]:
client.inference.put_model(
    task_type="text_embedding",
    inference_id="cohere_embeddings",
    body={
        "service": "cohere",
        "service_settings": {
            "api_key": COHERE_API_KEY,
            "model_id": "embed-english-v3.0",
            "embedding_type": "byte",
        },
    },
)

You can find your API keys in your Cohere dashboard under the
[API keys section](https://dashboard.cohere.com/api-keys).

## Create the index mapping

Create the index mapping for the index that will contain the embeddings.

In [ ]:
client.indices.create(
    index="characters-index",
    mappings={
        "properties": {
            "infer_field": {
                "type": "semantic_text",
                "inference_id": "cohere_embeddings",
            },
            "title": {"type": "text", "copy_to": "infer_field"},
        }
    },
)

## Prepare data and insert documents

This example uses the [SciFact](https://huggingface.co/datasets/mteb/scifact) data
set that you can find on HuggingFace.

In [19]:
url = "https://huggingface.co/datasets/gqd/fictional-characters/resolve/main/heros.json"

# Fetch the JSONL data from the URL
response = requests.get(url)
response.raise_for_status()  # Ensure we notice bad responses

# Split the content by new lines and parse each line as JSON
data = json.loads(response.text)

print(data[0])

# Prepare the documents to be indexed
documents = []
for line in data:
    data_dict = line
    documents.append(
        {
            "_index": "characters-index",
            "_source": data_dict,
        }
    )

# Use the bulk endpoint to index
helpers.bulk(client, documents)

print("Data ingestion completed, text embeddings generated!")

{'title': '"Eat" Owner', 'parsed': {'image': 'Vlcsnap-2016-09-06-05h31m34s50.png', 'origin': "''Wander Over Yonder''", 'fullname': 'Unknown', 'alias': '"Eat" Owner', 'occupation': 'Restaurant owner', 'skills': 'Management skills', 'goals': 'Manage his restaurant.<br>\nHelp the rebels defeat Lord Dominator.<br>', 'hobby': '', 'family': 'Unknown', 'friends': 'Michelle', 'enemies': 'Lord Dominator', 'type of hero': 'Alien'}, 'origin': "''Wander Over Yonder''"}


NameError: name 'client' is not defined

Your index is populated with the SciFact data and text embeddings for the text
field.

## Add the rerank inference endpoint

To combine the results more effectively, use 
[Cohere's Rerank v3](https://docs.cohere.com/docs/rerank-2) model through the
inference API to provide a more precise semantic reranking of the results.

Create an inference endpoint with your Cohere API key and the used model name as
the `model_id` (`rerank-english-v3.0` in this example).

In [ ]:
client.inference.put_model(
    task_type="rerank",
    inference_id="cohere_rerank",
    body={
        "service": "cohere",
        "service_settings": {
            "api_key": COHERE_API_KEY,
            "model_id": "rerank-english-v3.0",
        },
        "task_settings": {
            "top_n": 100,
        },
    },
)

Rerank the results using the new inference endpoint.

## Semantic search with reranking

Let's start querying the index!

In [ ]:
def semantic_search_with_reranking(query):
    return client.search(
        index="characters-index",
        retriever={
            "text_similarity_reranker": {
                "retriever": {
                    "standard": {
                        "query": {"semantic": {"field": "infer_field", "query": query}}
                    }
                },
                "field": "titles",
                "inference_id": "my_cohere_rerank_endpoint",
                "inference_text": query,
                "rank_window_size": 100,
            }
        },
    )

In [ ]:
pretty_print_characters(
    semantic_search_with_reranking("Who are the stars of the Harry Potter series?")
)
pretty_print_characters(
    semantic_search_with_reranking("Who are the hobbits in the lord of the rings?")
)
pretty_print_characters(
    semantic_search_with_reranking("Who are the hobbits in the hobbit?")
)
pretty_print_characters(
    semantic_search_with_reranking("Who are the stars of star wars")
)
pretty_print_characters(semantic_search_with_reranking("Who is the coolest character?"))

## Retrieval Augmented Generation (RAG) with Cohere and Elasticsearch

RAG is a method for generating text using additional information fetched from an
external data source. With the ranked results, you can build a RAG system on
top of what you created with 
[Cohere's Chat API](https://docs.cohere.com/docs/chat-api).

Pass in the retrieved documents and the query to receive a grounded response
using Cohere's newest generative model 
[Command R+](https://docs.cohere.com/docs/command-r-plus).

Then pass in the query and the documents to the Chat API, and print out the
response.

In [48]:
response = co.chat(message=query, documents=ranked_documents, model="command-r-plus")

source_documents = []
for citation in response.citations:
    for document_id in citation.document_ids:
        if document_id not in source_documents:
            source_documents.append(document_id)

print(f"Query: {query}")
print(f"Response: {response.text}")
print("Sources:")
for document in response.documents:
    if document["id"] in source_documents:
        print(f"{document['title']}: {document['text']}")

Query: What is biosimilarity?
Response: Biosimilarity is based on the comparability concept, which has been used successfully for several decades to ensure close similarity of a biological product before and after a manufacturing change. Over the last 10 years, experience with biosimilars has shown that even complex biotechnology-derived proteins can be copied successfully.
Sources:
Interchangeability of Biosimilars: A European Perspective: Many of the best-selling ‘blockbuster’ biological medicinal products are, or will soon be, facing competition from similar biological medicinal products (biosimilars) in the EU. Biosimilarity is based on the comparability concept, which has been used successfully for several decades to ensure close similarity of a biological product before and after a manufacturing change. Over the last 10 years, experience with biosimilars has shown that even complex biotechnology-derived proteins can be copied successfully. Most best-selling biologicals are used f